# Building Foursquare API
'''
In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. 
You will learn how to construct a URL to send a request to the API to search for a specific type of venues, 
to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get 
trending venues around a location. Also, you will learn how to use the visualization library, Folium, 
to visualize the results.
'''

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/nr/anaconda2

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    certifi-2018.11.29         |        py27_1000         145 KB  conda-forge
    conda-4.5.12               |        py27_1000         652 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         2.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0        conda-forge
    geopy:           1.18.1-

### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'KOPEIOPPWWFBAAYUQVKQ55WUSQJE3OOCEJ01W1CUCQI3Q1PF' # your Foursquare ID
CLIENT_SECRET = 'RVJ1RH220QUNK2JZNZMDO4UMWVYPBK4E43MEHCSESSB2GI4W' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KOPEIOPPWWFBAAYUQVKQ55WUSQJE3OOCEJ01W1CUCQI3Q1PF
CLIENT_SECRET:RVJ1RH220QUNK2JZNZMDO4UMWVYPBK4E43MEHCSESSB2GI4W


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In [17]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/Users/nr/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


(40.7149555, -74.0153365)


## 1. Search for a specific venue category
>`https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [18]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KOPEIOPPWWFBAAYUQVKQ55WUSQJE3OOCEJ01W1CUCQI3Q1PF&client_secret=RVJ1RH220QUNK2JZNZMDO4UMWVYPBK4E43MEHCSESSB2GI4W&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

#### Send the GET Request and examine the results

In [20]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5c1be25c6a607133f375ac1a'},
 u'response': {u'venues': [{u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       u'suffix': u'.png'},
      u'id': u'4bf58dd8d48988d1ca941735',
      u'name': u'Pizza Place',
      u'pluralName': u'Pizza Places',
      u'primary': True,
      u'shortName': u'Pizza'}],
    u'delivery': {u'id': u'294544',
     u'provider': {u'icon': {u'name': u'/delivery_provider_seamless_20180129.png',
       u'prefix': u'https://fastly.4sqi.net/img/general/cap/',
       u'sizes': [40, 50]},
      u'name': u'seamless'},
     u'url': u'https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544'},
    u'hasPerk': False,
    u'id': u'4fa862b3e4b0ebff2f749f06',
    u'location': {u'address': u'225 Murray St',
     u'cc': u'US',
     u'city': u'New York',
   

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{u'pluralName': u'Pizza Places', u'primary': ...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{u'lat': 40.7152177906, u'lng': -74.014739402...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1545331292
1,"[{u'pluralName': u'Food', u'primary': True, u'...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{u'lat': 40.71446, u'lng': -74.010086, u'labe...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1545331292
2,"[{u'pluralName': u'Italian Restaurants', u'pri...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{u'lat': 40.7153371386, u'lng': -74.008847662...",40.715337,-74.008848,10007,NY,Ecco,v-1545331292


#### Define information of interest and filter dataframe

In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{u'lat': 40.7152177906, u'lng': -74.014739402...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{u'lat': 40.71446, u'lng': -74.010086, u'labe...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{u'lat': 40.7153371386, u'lng': -74.008847662...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


#### Let's visualize the Italian restaurants that are nearby

In [23]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [25]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=KOPEIOPPWWFBAAYUQVKQ55WUSQJE3OOCEJ01W1CUCQI3Q1PF&client_secret=RVJ1RH220QUNK2JZNZMDO4UMWVYPBK4E43MEHCSESSB2GI4W&v=20180604'

#### Send GET request for result

In [28]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

[u'rating', u'reasons', u'colors', u'likes', u'beenHere', u'id', u'createdAt', u'hasMenu', u'verified', u'shortUrl', u'menu', u'hereNow', u'allowMenuUrlEdit', u'pageUpdates', u'contact', u'location', u'dislike', u'bestPhoto', u'listed', u'canonicalUrl', u'price', u'delivery', u'photos', u'attributes', u'stats', u'tips', u'categories', u'hours', u'ok', u'name', u'url', u'specials', u'ratingColor', u'ratingSignals', u'popular', u'timeZone', u'inbox']


{u'allowMenuUrlEdit': True,
 u'attributes': {u'groups': [{u'count': 1,
    u'items': [{u'displayName': u'Price',
      u'displayValue': u'$$',
      u'priceTier': 2}],
    u'name': u'Price',
    u'summary': u'$$',
    u'type': u'price'},
   {u'count': 7,
    u'items': [{u'displayName': u'Credit Cards',
      u'displayValue': u'Yes (incl. American Express)'}],
    u'name': u'Credit Cards',
    u'summary': u'Credit Cards',
    u'type': u'payments'},
   {u'count': 1,
    u'items': [{u'displayName': u'Outdoor Seating', u'displayValue': u'Yes'}],
    u'name': u'Outdoor Seating',
    u'summary': u'Outdoor Seating',
    u'type': u'outdoorSeating'},
   {u'count': 8,
    u'items': [{u'displayName': u'Brunch', u'displayValue': u'Brunch'},
     {u'displayName': u'Lunch', u'displayValue': u'Lunch'},
     {u'displayName': u'Dinner', u'displayValue': u'Dinner'},
     {u'displayName': u'Happy Hour', u'displayValue': u'Happy Hour'}],
    u'name': u'Menus',
    u'summary': u'Happy Hour, Brunch & more',

### B. Get the venue's overall rating

In [29]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Since this restaurant has no ratings, let's check the third restaurant.

In [31]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

In [33]:
result['response']['venue']['tips']['count']

16

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [34]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5c1be4bedb04f53ab6236fb1'},
 u'response': {u'tips': {u'count': 16,
   u'items': [{u'agreeCount': 2,
     u'authorInteractionType': u'liked',
     u'canonicalUrl': u'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     u'createdAt': 1521601350,
     u'disagreeCount': 0,
     u'id': u'5ab1cb46c9a517174651d3fe',
     u'lang': u'en',
     u'likes': {u'count': 0, u'groups': []},
     u'logView': True,
     u'text': u'A+ Italian food! Trust me on this: my mom\u2019s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     u'todo': {u'count': 0},
     u'type': u'user',
     u'user': {u'firstName': u'Nick',
      u'gender': u'male',
      u'id': u'484542633',
      u'lastName': u'El-Tawil',
      u'photo': {u'prefix': u'https://fastly.4sqi.net/img/user/',
       u'suffix': u'/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'

#### Get tips and list of associated features

In [35]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

[u'lang',
 u'todo',
 u'canonicalUrl',
 u'agreeCount',
 u'text',
 u'logView',
 u'disagreeCount',
 u'authorInteractionType',
 u'likes',
 u'type',
 u'id',
 u'createdAt',
 u'user']

#### Format column width and display all tips

In [37]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",2,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [38]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

[u'checkins',
 u'bio',
 u'gender',
 u'type',
 u'canonicalUrl',
 u'firstName',
 u'lastName',
 u'lists',
 u'homeCity',
 u'photos',
 u'contact',
 u'photo',
 u'tips',
 u'mayorships',
 u'friends',
 u'id',
 u'lenses']

In [39]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


#### How many tips has this user submitted?

In [41]:
user_data['tips']

{u'count': 246}

Wow! So it turns out that Nick is a very active Foursquare user, with more than 240 tips.

### Get User's tips

In [42]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,"Great for a quick, cheap bite. I stop by when I’m in the neighborhood and don’t have much time to eat.",1,0,5acbec0c6fa81f1967dfeba3
8,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
9,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,1,0,5acbbcda01235808d5d6dc75


#### Let's get the venue for the tip with the greatest number of agree counts

In [43]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{u'city': u'New York', u'labeledLatLngs': [{u'lat': 40.73375031678678, u'lng': -74.00628236760791, u'label': u'display'}], u'cc': u'US', u'country': u'United States', u'postalCode': u'10014', u'state': u'NY', u'formattedAddress': [u'519 Hudson St (at W 10th St)', u'New York, NY 10014', u'United States'], u'crossStreet': u'at W 10th St', u'address': u'519 Hudson St', u'lat': 40.73375031678678, u'lng': -74.00628236760791}


### Get User's friends

In [44]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

In [45]:
### Retrieve the User's Profile Image

In [46]:
user_data

{u'bio': u'https://www.tawil.team/nick-el-tawil/',
 u'canonicalUrl': u'https://foursquare.com/nickeltawil',
 u'checkins': {u'count': 1, u'items': []},
 u'contact': {},
 u'firstName': u'Nick',
 u'friends': {u'count': 0,
  u'groups': [{u'count': 0,
    u'items': [],
    u'name': u'Other friends',
    u'type': u'others'}]},
 u'gender': u'male',
 u'homeCity': u'New York, NY',
 u'id': u'484542633',
 u'lastName': u'El-Tawil',
 u'lenses': [],
 u'lists': {u'count': 2,
  u'groups': [{u'count': 0, u'items': [], u'type': u'created'},
   {u'count': 0, u'items': [], u'type': u'followed'},
   {u'count': 2,
    u'items': [{u'canonicalUrl': u'https://foursquare.com/nickeltawil/list/todos',
      u'collaborative': False,
      u'description': u'',
      u'editable': False,
      u'id': u'484542633/todos',
      u'listItems': {u'count': 0},
      u'name': u"Nick's Saved Places",
      u'public': True,
      u'type': u'todos',
      u'url': u'/nickeltawil/list/todos'},
     {u'canonicalUrl': u'https://fo

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [48]:
latitude = 40.715337
longitude = -74.008848

In [49]:
#### Define URL

In [50]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=KOPEIOPPWWFBAAYUQVKQ55WUSQJE3OOCEJ01W1CUCQI3Q1PF&client_secret=RVJ1RH220QUNK2JZNZMDO4UMWVYPBK4E43MEHCSESSB2GI4W&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

In [51]:
#### Send GET request and examine results

In [52]:
import requests

In [53]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [54]:
#### Get relevant part of JSON

In [55]:
items = results['response']['groups'][0]['items']
items[0]

{u'reasons': {u'count': 0,
  u'items': [{u'reasonName': u'globalInteractionReason',
    u'summary': u'This spot is popular',
    u'type': u'general'}]},
 u'referralId': u'e-0-4af5d65ff964a52091fd21e3-0',
 u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     u'suffix': u'.png'},
    u'id': u'4bf58dd8d48988d1f8941735',
    u'name': u'Furniture / Home Store',
    u'pluralName': u'Furniture / Home Stores',
    u'primary': True,
    u'shortName': u'Furniture / Home'}],
  u'id': u'4af5d65ff964a52091fd21e3',
  u'location': {u'address': u'57 Warren St',
   u'cc': u'US',
   u'city': u'New York',
   u'country': u'United States',
   u'crossStreet': u'Church St',
   u'distance': 73,
   u'formattedAddress': [u'57 Warren St (Church St)',
    u'New York, NY 10007',
    u'United States'],
   u'labeledLatLngs': [{u'label': u'display',
     u'lat': 40.71482437714839,
     u'lng': -74.00940425461492}],
   u'lat': 40.71482437714839,
   u'lng': -

#### Process JSON and convert it to a clean dataframe

In [57]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{u'lat': 40.7148243771, u'lng': -74.0094042546, u'label': u'display'}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{u'lat': 40.7157730639, u'lng': -74.0097182331, u'label': u'display'}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
2,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,179,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{u'lat': 40.7137232263, u'lng': -74.0087326697, u'label': u'display'}]",40.713723,-74.008733,NaN,10007,NY,4c154c9a77cea593c401d260
3,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{u'lat': 40.7165264741, u'lng': -74.0081010847, u'label': u'display'}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
4,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{u'lat': 40.7147876991, u'lng': -74.0111317502, u'label': u'display'}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
5,Nish Nūsh,Falafel Restaurant,88 Reade St,US,New York,United States,at Church St,97,"[88 Reade St (at Church St), New York, NY 10013, United States]","[{u'lat': 40.7155371012, u'lng': -74.0077245293, u'label': u'display'}]",40.715537,-74.007725,NaN,10013,NY,50ba9119e4b071a4bae6dc10
6,Mulberry & Vine,Café,73 Warren St,US,New York,United States,btwn W Broadway & Greenwich St,117,"[73 Warren St (btwn W Broadway & Greenwich St), New York, NY 10007, United States]","[{u'lat': 40.7151769397, u'lng': -74.0102274778, u'label': u'display'}]",40.715177,-74.010227,NaN,10007,NY,5171b5cc011cef9833bbb787
7,Ten Over Ten,Nail Salon,112 Reade St,US,New York,United States,NaN,68,"[112 Reade St, New York, NY 10013, United States]","[{u'lat': 40.7159412557, u'lng': -74.0087205356, u'label': u'display'}]",40.715941,-74.008721,NaN,10013,NY,4ce2a27cd58c60fc0fa1a76f
8,Equinox Tribeca,Gym,54 Murray Street,US,New York,United States,at W Broadway,159,"[54 Murray Street (at W Broadway), New York, NY 10007, United States]","[{u'lat': 40.714057, u'lng': -74.009694, u'label': u'display'}]",40.714057,-74.009694,NaN,10007,NY,4a6e331af964a52031d41fe3
9,Little Park,American Restaurant,85 W Broadway,US,New York,United States,at Chambers St,29,"[85 W Broadway (at Chambers St), New York, NY 10007, United States]","[{u'lat': 40.7154865852, u'lng': -74.0091331351, u'label': u'display'}]",40.715487,-74.009133,NaN,10007,NY,545c0436498e798e22ce4b2a


#### Let's visualize these items on the map around our location

In [58]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.

In [59]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5c1bee234c1f677d295c61dc'},
 u'response': {u'venues': []}}

In [60]:
### Check if any venues are trending at this time

In [66]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [67]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

In [68]:
### Visualize trending venues

In [69]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [70]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'